# EDA: Data Pipeline Challenge

Exploratory analysis of the three sources: Google Ads (JSON), Facebook export (CSV), CRM revenue (CSV).
Goals: flatten Google Ads JSON, inspect shapes/dtypes, identify join keys and data quality issues.

In [1]:
import json
import pandas as pd
from pathlib import Path

DATA_DIR = Path("data")

## 1. Google Ads (JSON) — flatten to table

Structure: `campaigns[]` each with `daily_metrics[]`. We flatten to one row per campaign per date.

In [2]:
with open(DATA_DIR / "google_ads_api.json") as f:
    google_raw = json.load(f)

rows = []
for camp in google_raw["campaigns"]:
    for m in camp["daily_metrics"]:
        rows.append({
            "campaign_id": camp["campaign_id"],
            "campaign_name": camp["campaign_name"],
            "campaign_type": camp["campaign_type"],
            "status": camp["status"],
            "date": m["date"],
            "impressions": m["impressions"],
            "clicks": m["clicks"],
            "cost_micros": m["cost_micros"],
            "conversions": m["conversions"],
            "conversion_value": m["conversion_value"],
        })

df_google = pd.DataFrame(rows)
df_google["platform"] = "google_ads"
df_google

,campaign_id,campaign_name,campaign_type,status,date,impressions,clicks,cost_micros,conversions,conversion_value,platform
0,goog_camp_001,Brand Awareness - Q1,DISPLAY,ENABLED,2024-01-01,45230,892,125340000,23,2875.50,google_ads
1,goog_camp_001,Brand Awareness - Q1,DISPLAY,ENABLED,2024-01-02,48120,956,134560000,28,3520.00,google_ads
2,goog_camp_001,Brand Awareness - Q1,DISPLAY,ENABLED,2024-01-03,42890,834,118230000,19,2280.00,google_ads
3,goog_camp_001,Brand Awareness - Q1,DISPLAY,ENABLED,2024-01-04,51230,1023,145670000,31,3875.25,google_ads
4,goog_camp_001,Brand Awareness - Q1,DISPLAY,ENABLED,2024-01-05,38920,756,108450000,17,1955.00,google_ads
5,goog_camp_001,Brand Awareness - Q1,DISPLAY,ENABLED,2024-01-06,29340,567,78230000,12,1440.00,google_ads
6,goog_camp_001,Brand Awareness - Q1,DISPLAY,ENABLED,2024-01-07,31250,612,85670000,14,1680.00,google_ads
7,goog_camp_001,Brand Awareness - Q1,DISPLAY,ENABLED,2024-01-08,47890,945,132450000,26,3120.00,google_ads
8,goog_camp_001,Brand Awareness - Q1,DISPLAY,ENABLED,2024-01-09,52340,1067,149870000,33,4125.75,google_ads
9,goog_camp_001,Brand Awareness - Q1,DISPLAY,ENABLED,2024-01-10,49120,978,138920000,29,3625.00,google_ads


In [3]:
df_google.info()
df_google.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   campaign_id       45 non-null     object 
 1   campaign_name     45 non-null     object 
 2   campaign_type     45 non-null     object 
 3   status            45 non-null     object 
 4   date              45 non-null     object 
 5   impressions       45 non-null     int64  
 6   clicks            45 non-null     int64  
 7   cost_micros       45 non-null     int64  
 8   conversions       45 non-null     int64  
 9   conversion_value  45 non-null     float64
 10  platform          45 non-null     object 
dtypes: float64(1), int64(4), object(6)
memory usage: 4.0+ KB


,impressions,clicks,cost_micros,conversions,conversion_value
count,45.000000,45.000000,4.500000e+01,45.000000,45.000000
mean,26035.333333,950.111111,1.431091e+08,51.400000,6839.366667
std,14005.170084,429.682400,7.460206e+07,30.621888,4434.782235
min,8560.000000,356.000000,4.867000e+07,11.000000,1320.000000
25%,14230.000000,612.000000,8.567000e+07,26.000000,3250.000000
50%,24230.000000,892.000000,1.253400e+08,47.000000,5875.000000
75%,31250.000000,1098.000000,1.774500e+08,65.000000,8820.000000
max,53240.000000,1876.000000,3.034500e+08,118.000000,16520.000000


## 2. Facebook export (CSV)

In [4]:
df_facebook = pd.read_csv(DATA_DIR / "facebook_export.csv")
df_facebook["platform"] = "facebook"
df_facebook.head(10)

,campaign_id,campaign_name,date,impressions,clicks,spend,purchases,purchase_value,reach,frequency,platform
0,fb_camp_001,Spring Collection Awareness,2024-01-01,78450,1234,145.67,34.0,4250.0,65000,1.21,facebook
1,fb_camp_001,Spring Collection Awareness,2024-01-02,82340,1345,156.89,42.0,5250.0,68000,1.21,facebook
2,fb_camp_001,Spring Collection Awareness,01/03/2024,76890,1189,138.45,31.0,3875.0,63500,1.21,facebook
3,fb_camp_001,Spring Collection Awareness,2024-01-04,89120,1456,167.23,48.0,6000.0,73500,1.21,facebook
4,fb_camp_001,Spring Collection Awareness,2024-01-05,71230,1098,129.34,28.0,3500.0,59000,1.21,facebook
5,fb_camp_001,Spring Collection Awareness,2024-01-06,52340,798,94.56,19.0,2375.0,43500,1.20,facebook
6,fb_camp_001,Spring Collection Awareness,2024-01-07,54670,834,99.12,21.0,2625.0,45500,1.20,facebook
7,fb_camp_001,Spring Collection Awareness,2024-01-08,84560,1378,159.34,45.0,5625.0,70000,1.21,facebook
8,fb_camp_001,Spring Collection Awareness,2024-01-09,91230,1502,175.67,52.0,6500.0,75500,1.21,facebook
9,fb_camp_001,Spring Collection Awareness,01-10-2024,87450,1423,163.89,47.0,5875.0,72500,1.21,facebook


In [5]:
df_facebook.info()
df_facebook["date"].value_counts(dropna=False).head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   campaign_id     45 non-null     object 
 1   campaign_name   45 non-null     object 
 2   date            45 non-null     object 
 3   impressions     45 non-null     int64  
 4   clicks          45 non-null     int64  
 5   spend           45 non-null     float64
 6   purchases       43 non-null     float64
 7   purchase_value  45 non-null     float64
 8   reach           45 non-null     int64  
 9   frequency       45 non-null     float64
 10  platform        45 non-null     object 
dtypes: float64(4), int64(3), object(4)
memory usage: 4.0+ KB


date
2024-01-01    3
2024-01-04    3
2024-01-05    3
2024-01-06    3
2024-01-08    3
2024-01-09    3
2024-01-02    3
2024-01-11    3
2024-01-13    3
2024-01-14    3
2024-01-15    3
2024-01-10    2
2024-01-07    2
2024-01-12    2
2024-01-03    2
Name: count, dtype: int64

## 3. CRM revenue (CSV)

In [6]:
# Note: one row has date "January 4, 2024" (unquoted comma) — use engine='python' to avoid parse error
df_crm = pd.read_csv(DATA_DIR / "crm_revenue.csv", engine="python", on_bad_lines="warn")
df_crm.head(10)

/var/folders/zc/k23mrjxx77j1ml1ck79z_p7h0000gn/T/ipykernel_91058/3186257508.py:2: ParserWarning: Skipping line 20: Expected 8 fields in line 20, saw 9

  df_crm = pd.read_csv(DATA_DIR / "crm_revenue.csv", engine="python", on_bad_lines="warn")


,order_id,customer_id,order_date,revenue,channel_attributed,campaign_source,product_category,region
0,ORD-10001,CUST-5234,2024-01-01,125.50,google,goog_camp_002,Electronics,North America
1,ORD-10002,CUST-8921,2024-01-01,89.99,facebook,fb_camp_002,Apparel,Europe
2,ORD-10003,CUST-3456,01/01/2024,234.00,google,goog_camp_001,Home & Garden,North America
3,ORD-10004,CUST-7823,2024-01-01,67.50,facebook,fb_camp_001,Apparel,North America
4,ORD-10005,CUST-2341,2024-01-01,445.00,google,goog_camp_002,Electronics,Europe
5,ORD-10006,CUST-9012,2024-01-02,178.25,facebook,fb_camp_002,Home & Garden,North America
6,ORD-10007,CUST-4567,2024-01-02,56.00,google,goog_camp_003,Apparel,Asia Pacific
7,ORD-10008,CUST-6789,2024-01-02,NaN,facebook,fb_camp_001,Electronics,North America
8,ORD-10009,CUST-1234,2024-01-02,312.75,google,goog_camp_002,Electronics,Europe
9,ORD-10010,CUST-8456,2024-01-02,89.00,facebook,fb_camp_002,Apparel,North America


In [7]:
df_crm.info()
df_crm["channel_attributed"].value_counts(dropna=False)
# Note: order_date may have multiple formats (e.g. 01/01/2024, January 4 2024)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   order_id            84 non-null     object 
 1   customer_id         83 non-null     object 
 2   order_date          84 non-null     object 
 3   revenue             83 non-null     float64
 4   channel_attributed  84 non-null     object 
 5   campaign_source     83 non-null     object 
 6   product_category    84 non-null     object 
 7   region              84 non-null     object 
dtypes: float64(1), object(7)
memory usage: 5.4+ KB


channel_attributed
google      41
facebook    41
Google       1
FACEBOOK     1
Name: count, dtype: int64

## 4. Join keys and relationships

- **Ad performance (Google + Facebook)** ↔ **CRM orders**: link by `campaign_id` (e.g. `goog_camp_002`, `fb_camp_001`) and date. CRM has `campaign_source` = campaign_id and `channel_attributed` = google/facebook.
- **Google** and **Facebook** can be combined on `campaign_id` + `date` (with a `platform` column) for unified ad metrics.
- CRM `order_date` aligns with ad `date` for attribution (same day or date range).

In [8]:
print("Google campaign_ids:", df_google["campaign_id"].unique().tolist())
print("Facebook campaign_ids:", df_facebook["campaign_id"].unique().tolist())
print("CRM campaign_source (sample):", df_crm["campaign_source"].dropna().unique()[:10].tolist())
print("CRM channel_attributed:", df_crm["channel_attributed"].dropna().unique().tolist())

Google campaign_ids: ['goog_camp_001', 'goog_camp_002', 'goog_camp_003']
Facebook campaign_ids: ['fb_camp_001', 'fb_camp_002', 'fb_camp_003']
CRM campaign_source (sample): ['goog_camp_002', 'fb_camp_002', 'goog_camp_001', 'fb_camp_001', 'goog_camp_003', 'fb_camp_003']
CRM channel_attributed: ['google', 'facebook', 'Google', 'FACEBOOK']


## 5. Data quality snapshot

Quick checks: nulls, duplicates, date formats, invalid values.

In [9]:
print("=== Google Ads ===")
print(df_google.isnull().sum())
print("Duplicates (campaign_id + date):", df_google.duplicated(subset=["campaign_id", "date"]).sum())
print()
print("=== Facebook ===")
print(df_facebook.isnull().sum())
print("Duplicates (campaign_id + date):", df_facebook.duplicated(subset=["campaign_id", "date"]).sum())
print()
print("=== CRM ===")
print(df_crm.isnull().sum())
print("Duplicates (order_id):", df_crm.duplicated(subset=["order_id"]).sum())
if "revenue" in df_crm.columns:
    print("Revenue min:", df_crm["revenue"].min(), "max:", df_crm["revenue"].max())

=== Google Ads ===
campaign_id         0
campaign_name       0
campaign_type       0
status              0
date                0
impressions         0
clicks              0
cost_micros         0
conversions         0
conversion_value    0
platform            0
dtype: int64
Duplicates (campaign_id + date): 0

=== Facebook ===
campaign_id       0
campaign_name     0
date              0
impressions       0
clicks            0
spend             0
purchases         2
purchase_value    0
reach             0
frequency         0
platform          0
dtype: int64
Duplicates (campaign_id + date): 0

=== CRM ===
order_id              0
customer_id           1
order_date            0
revenue               1
channel_attributed    0
campaign_source       1
product_category      0
region                0
dtype: int64
Duplicates (order_id): 3
Revenue min: -50.0 max: 9999999.99


## 6. Column-by-column discrepancies

Below we loop through each dataset and each column to surface: nulls, date format variety, duplicate keys, numeric issues (negatives, outliers), and categorical consistency. Use this to define validation rules.

In [11]:
def date_format_counts(ser):
    """Count distinct string representations (for date columns)."""
    if ser.isnull().all():
        return {}
    return ser.astype(str).value_counts(dropna=False).to_dict()

def check_column(df, col):
    """Print discrepancy checks for one column."""
    print(f"  [{col}] dtype={df[col].dtype}")
    # Count distinct value types in this column (e.g. int, float, str)
    value_types = set(type(v).__name__ for v in df[col])
    print(f"    → Distinct value types: {len(value_types)} ({', '.join(sorted(value_types))})")
    nulls = df[col].isnull().sum()
    if nulls > 0:
        print(f"    → Nulls: {nulls}")
    # Date columns: show format variety
    if "date" in col.lower() or col == "order_date":
        fmts = date_format_counts(df[col])
        if len(fmts) > 1:
            print(f"    → Date formats: {len(fmts)} distinct")
            for fmt, cnt in list(fmts.items())[:8]:
                print(f"       '{fmt}': {cnt}")
        else:
            print(f"    → Date format: 1 (OK)")
    # Numeric: min/max and flags
    if df[col].dtype in ("int64", "float64") or pd.api.types.is_numeric_dtype(df[col]):
        num = pd.to_numeric(df[col], errors="coerce")
        if num.notna().any():
            print(f"    → Min: {num.min():.2f}, Max: {num.max():.2f}")
            if (num < 0).any():
                print(f"    → Negative values: {(num < 0).sum()}")
            if col == "revenue" and num.notna().any():
                q99 = num.quantile(0.99)
                if (num > q99 * 10).any():
                    print(f"    → Possible outliers (>{q99:.0f}*10): {(num > q99*10).sum()}")
    # Categorical (small number of unique)
    if col == "channel_attributed" or col == "campaign_source":
        uniq = df[col].dropna().astype(str).str.strip().unique()
        if len(uniq) <= 20:
            print(f"    → Values: {list(uniq)}")

datasets = [
    ("Google Ads", df_google, ["campaign_id", "date"]),
    ("Facebook", df_facebook, ["campaign_id", "date"]),
    ("CRM", df_crm, ["order_id"]),
]
for name, df, key_cols in datasets:
    print("\n" + "="*60)
    print(name)
    print("="*60)
    for col in df.columns:
        check_column(df, col)
    # Key duplicate summary
    dup = df.duplicated(subset=key_cols).sum()
    print(f"\n  [Key duplicates ({key_cols})]: {dup}")
print("\nDone.")


Google Ads
  [campaign_id] dtype=object
    → Distinct value types: 1 (str)
  [campaign_name] dtype=object
    → Distinct value types: 1 (str)
  [campaign_type] dtype=object
    → Distinct value types: 1 (str)
  [status] dtype=object
    → Distinct value types: 1 (str)
  [date] dtype=object
    → Distinct value types: 1 (str)
    → Date formats: 15 distinct
       '2024-01-01': 3
       '2024-01-02': 3
       '2024-01-03': 3
       '2024-01-04': 3
       '2024-01-05': 3
       '2024-01-06': 3
       '2024-01-07': 3
       '2024-01-08': 3
  [impressions] dtype=int64
    → Distinct value types: 1 (int)
    → Min: 8560.00, Max: 53240.00
  [clicks] dtype=int64
    → Distinct value types: 1 (int)
    → Min: 356.00, Max: 1876.00
  [cost_micros] dtype=int64
    → Distinct value types: 1 (int)
    → Min: 48670000.00, Max: 303450000.00
  [conversions] dtype=int64
    → Distinct value types: 1 (int)
    → Min: 11.00, Max: 118.00
  [conversion_value] dtype=float64
    → Distinct value types: 1 (